<a href="https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/ACNS0332/ACNS0332.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing DICOM images and annotations from the ACNS0332 dataset hosted on TCIA

This notebook is focused on accessing the **"Annotations for Chemotherapy and Radiation Therapy in Treating Young Patients With Newly Diagnosed, Previously Untreated, High-Risk Medulloblastoma/PNET (ACNS0332)"** dataset hosted on [The Cancer Imaging Archive(TCIA)](https://cancerimagingarchive.net). 


# 1 Learn about and request access to the datasets

The images, annotations (tumor segmentation and seed point labels), and clinical data associated with this trial are described in detail at the following links.  These pages are publicly visible without logging in, and can be used to obtain an understanding of the dataset before going through the trouble of requesting access.

1.  [Image Collection Summary](https://doi.org/10.7937/TCIA.582B-XZ89)
2.  [Annotation Summary](https://doi.org/10.7937/D8A8-6252)
3.  [Clinical datasets](https://nctn-data-archive.nci.nih.gov/node/838)

**Note:** You can use the links above to view data dictionaries outlining the specific clinical variables that were collected before requesting access.

### Requesting access to the data
To download the actual data, you must request access through the NCTN Data Archive via the following steps:
 
 1. [Register an account on the NCTN Data Archive](https://nctn-data-archive.nci.nih.gov/).  
 2. After logging in, use the "Request Data" link in the left side menu.  
 3. Follow the on-screen instructions and enter _**NCT00392327**_ when asked which trial you want to request.  
 4. In step 2 of the Create Request form, be sure to select “Imaging Data Requested”. 
 
Once you are approved for access, you'll be able to download the clinical data from the NCTN Archive.  You will then be asked to create an account on TCIA with the same email address to access the imaging data.  Please contact NCINCTNDataArchive@mail.nih.gov for any questions about access requests.  

# 2 Import tcia_utils

The following imports a variety of useful functions for accessing TCIA via Jupyter/Python.

In [ ]:
import requests
import pandas as pd

tcia_utils_text = requests.get("https://github.com/kirbyju/TCIA_Notebooks/raw/main/tcia_utils.py")
with open('tcia_utils.py', 'wb') as f:
    f.write(tcia_utils_text.content)

In [ ]:
import tcia_utils as tcia

# 3 Downloading images and annotations with the NBIA Data Retriever

TCIA uses software called NBIA to manage its DICOM data.  One way to download TCIA data is to install the [linux command-line version of the NBIA Data Retriever](https://wiki.cancerimagingarchive.net/x/2QKPBQ) using the following steps.  This tool provides a number of useful features such as auto-retry if there are any problems, saving data in an organized hierarchy on your hard drive (Collection > Patient > Study > Series > Images), and providing a CSV file containing key DICOM metadata about the images you've downloaded.

_**Note:**_ It's also possible to download these data via our REST API if you can't or don't want to install the NBIA Data Retriever. This is covered later in the notebook.

## 3.1 Install the NBIA Data Retriever CLI package

In [ ]:
# Install NBIA Data Retriever CLI software for downloading images later in this notebook.

!mkdir /usr/share/desktop-directories/
!wget -P /content/NBIA-Data-Retriever https://cbiit-download.nci.nih.gov/nbia/releases/ForTCIA/NBIADataRetriever_4.4/nbia-data-retriever-4.4.deb
!dpkg -i /content/NBIA-Data-Retriever/nbia-data-retriever-4.4.deb

# NOTE: If you're working on a Linux OS that uses RPM packages, you can change the wget line above to point to
#       https://cbiit-download.nci.nih.gov/nbia/releases/ForTCIA/NBIADataRetriever_4.4/NBIADataRetriever-4.4-1.x86_64.rpm

## 3.2 Set up your credential file
Since this Collection requires logging in, you must setup a **credentials.txt** file that contains your user name and password. We will leverage the **tcia.makeCredentialFile()** function to do this.

In [ ]:
tcia.makeCredentialFile()

## 3.3 Download data
The NBIA Data Retriever software works by ingesting a "manifest" file that contains the DICOM Series Instance UIDs of the scans you want to download. The manifest files can be downloaded from [this page](https://doi.org/10.7937/D8A8-6252), but you can also obtain these manifests with the commands below.

* Annotations - segmentations, seed points, and Negative Findings Assessments 
* Source images used to create segmentations & seed points
* Source images used to create Negative Assessment Reports

In [ ]:
# Annotations - segmentations, seed points, and Negative Findings Assessments
manifest = requests.get("https://wiki.cancerimagingarchive.net/download/attachments/119703167/ACNS0332-Tumor-Annotations-manifest_10-04-2022.tcia?api=v2")
with open('ACNS0332-Tumor-Annotations-manifest_10-04-2022.tcia', 'wb') as f:
    f.write(manifest.content)

In [ ]:
# Source images used to create segmentations & seed points
manifest = requests.get("https://wiki.cancerimagingarchive.net/download/attachments/119703167/ACNS0332-OriginalMRs-SEGSandSeedpoints-manifest_10-04-2022.tcia?api=v2")
with open('ACNS0332-OriginalMRs-SEGSandSeedpoints-manifest_10-04-2022.tcia', 'wb') as f:
    f.write(manifest.content)


In [ ]:
# Source images used to create Negative Assessment Reports
# (no segmentation or seed points created for the scan)
manifest = requests.get("https://wiki.cancerimagingarchive.net/download/attachments/119703167/ACNS0332-OriginalMRs-NegativeAssessments-manifest_10-04-2022.tcia?api=v2")
with open('ACNS0332-OriginalMRs-NegativeAssessments-manifest_10-04-2022.tcia', 'wb') as f:
    f.write(manifest.content)

A manifest containing sample images and annotations for a single subject has also been created for use with this notebook to facilitate quick testing and demonstrations.

In [ ]:
# Single subject manifest containing examples of each annotation type
# Use this one for a quick demo.
manifest = requests.get("https://github.com/kirbyju/TCIA_Notebooks/raw/main/ACNS0332/acns0332-demo-PARJIR.tcia")
with open('acns0332-demo-PARJIR.tcia', 'wb') as f:
    f.write(manifest.content)

Now we can open the sample manifest file with the NBIA Data Retriever to download the actual data. You can repeat this step for each manifest you want to download by changing the path.

**<font color='red'>After running the following command, click in the output cell, type "y," and press Enter to agree with the TCIA Data Usage Policy and start the download.</font>**

In [ ]:
!/opt/nbia-data-retriever/nbia-data-retriever --cli '/content/acns0332-demo-PARJIR.tcia' -d /content/ -l /content/credentials.txt

# 4 Accessing the REST APIs 
The NBIA REST APIs allow TCIA users to query metadata and download image data.  Since this dataset is "limited access," we'll need to use the "NBIA Search with Authentication REST API" described at https://wiki.cancerimagingarchive.net/x/X4ATBg, which enables you to use your login credentials to create API tokens to access this Collection.  However, we'll rely heavily on [tcia_utils](https://github.com/kirbyju/TCIA_Notebooks/blob/main/tcia_utils.py) to simplify things.  

## 4.1 Create an API token

We'll use **tcia.getToken()** to generate an access token to query restricted Collections on TCIA.  **<font color='red'>Tokens are valid for 2 hours and must be refreshed after that point.</font>** See https://wiki.cancerimagingarchive.net/x/X4ATBg for more details. 

In [ ]:
access_token = tcia.getToken()

## 4.2 Explore the data with REST API queries

Let's start by looking at what body parts and modalities are contained in the Collection.  For this dataset, the 3D segmentations are SEG modality. RTSTRUCTs were used to record seed points and scans where no tumor was found.

In [ ]:
# count patients for each modality
data = tcia.getModalityCounts(collection = "ACNS0332")

# format results in dataframe
df = pd.DataFrame(data)
df.rename(columns = {'criteria':'Modality', 'count':'PatientCount'}, inplace = True)
df.PatientCount = df.PatientCount.astype(int)
display(df.sort_values(by='PatientCount', ascending=False, ignore_index = True))

In [ ]:
# Count patients for each body part examined
data = tcia.getBodyPartCounts(collection = "ACNS0332")

# format results in dataframe
df = pd.DataFrame(data)
df.rename(columns = {'criteria':'BodyPartExamined', 'count':'PatientCount'}, inplace = True)
df.PatientCount = df.PatientCount.astype(int)
display(df.sort_values(by='PatientCount', ascending=False, ignore_index = True))

Now let's run some queries to see what we can learn about the patient cohort from the DICOM metadata.  This information can include things like age, gender, and ethnicity.  However, in the case of ACNS0332, most of this information is also available in the clinical data at https://nctn-data-archive.nci.nih.gov/node/838.

In [ ]:
# obtain patient details (e.g. species, gender, ethnicity) for the collection 
data = tcia.getPatient(collection = "ACNS0332", api_url = "restricted")

# format as dataframe
df = pd.DataFrame(data)
display(df)

# optional - save to CSV file
# df.to_csv('patient_metadata.csv')

In [ ]:
# obtain study/visit details (e.g. anonymized study date, age at the time of visit)
data = tcia.getStudy(collection = "ACNS0332", api_url = "restricted")

# format as dataframe
df = pd.DataFrame(data)
display(df)

# optional - save to CSV file
# df.to_csv('study_metadata.csv')

We can also create a report that gives useful metadata about each scan in the dataset (e.g. series description, modality, scanner manufacturer & software version, number of images).

In [ ]:
# obtain scan/series metadata for a collection as JSON
collection_series = tcia.getSeries(collection = "ACNS0332", api_url = "restricted")

# format as dataframe
df = pd.DataFrame(collection_series)
display(df)

# optional - save to CSV file
# df.to_csv('series_metadata.csv')

Finally, we can use the results from the getSeries() query to generate some summary statistics about the scans in the collection.

In [ ]:
# Calculate summary statistics for a given collection 
tcia.makeSeriesReport(collection_series)

## 4.3 Downloading data with the REST API
Next we'll cover using the API to download data.  This can be useful if you'd like to download results from API queries rather than using an existing manifest file.  It's also useful if you can't install the NBIA Data Retriever or want to integrate TCIA downloads into other pipelines/tools.  

As a reminder, many of the scans in the ACNS0332 Collection were not annotated by the authors of https://doi.org/10.7937/D8A8-6252.  The reasons are outlined in the Annotation Protocol on that page.  As a result, you may want to download only a subset of the scans such as:

1. Seed point labels (RTSTRUCTs)
2. 3d segmentation labels (SEGs)
3. All source images used to create either seed points or segmentations
4. Source images used to create segmentations
5. Source images with negative finding assessments

The following examples demonstrate how to tackle each of these use cases. 

### 4.3.1 tcia_utils download functions
**tcia_utils** contains two functions for downloading data, **downloadSampleSeries()** and **downloadSeries()**.   The only difference between them is that **downloadSampleSeries()** only grabs the first three scans in the list of scans to download, which is useful for demonstration and testing purposes.

Both functions ingest a set of seriesUids to download.  By default, the functions expect JSON data containing "SeriesInstanceUID" elements, which can be generated using **getSeries()**.  However, if you have a series UID list from some other source, you can set **input_type = "list"** to pass a regular list of series UIDs instead of JSON. Both functions return a dataframe of the series metadata describing the data that were downloaded.  You can optionally export a CSV of the series metadata by specifying the **csv_filename** parameter.

### 4.3.2 Download a sample subject
Before downloading data for the entire cohort, let's take a quick look at the various types of data for a single subject.  We'll use the **acns0332-demo-PARJIR.tcia** manifest file in this example to tell **tcia.downloadSeries()** what to download.

TCIA manifest files contain several lines of download parameters that precede a list of Series Instance UIDs to download.  The steps below will put the Series UIDs into a list and ignore the parameters in the first six lines of the manifest so that we have a clean list to feed to **tcia.downloadSeries()**.

In [ ]:
# initialize variable
series_data = []

# open file
with open("acns0332-demo-PARJIR.tcia") as f:
    for line in f:
        series_data.append(line.rstrip())

# remove the parameters from the list
del series_data[:6]
#print(series_data)

print("Result contains", len(series_data), "Series Instance UIDs (scans).")


Now we can download the series in the list and return the metadata to a dataframe. 

In [ ]:
df = tcia.downloadSeries(series_data, input_type = "list", api_url = "restricted")


Here's what the metadata looks like in the dataframe.

In [ ]:
display(df)

### 4.3.3 Download subsets of the data
To identify the subsets for the other use cases, we'll leverage the **annotation metadata** spreadsheet the authors provided, which you can download from https://doi.org/10.7937/D8A8-6252 or retrieve directly into a dataframe with the code below.

In [ ]:
# load annotation metadata spreadsheet to dataframe

annotation_Metadata = pd.read_csv('https://wiki.cancerimagingarchive.net/download/attachments/119703167/ACNS0332_annotations_metadata-2022-10-04.csv?api=v2')

display(annotation_Metadata)

#### Download seed points
Since we're working with Series UIDs from a dataframe instead of JSON output from the API, we'll use the  **input_type = "list"** parameter in the remaining download steps.  Options to download a sample (three scans) or the entire dataset are provided.  We'll also specify a **csv_filename** to save the related metadata to a file.

In [ ]:
# filter dataframe to only include seed point rows
seedPoints = annotation_Metadata[annotation_Metadata['StructureSetLabel'] == 'Seed Points']
#display(seedPoints)

# extract series UID column to list for downloading
series_data = seedPoints["SeriesInstanceUID"].tolist()

# download a sample set of three scans 
# return metadata dataframe as dataframe
# save a CSV of the metadata 
df = tcia.downloadSampleSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_seedPoints")

# If you'd rather download the full dataset, run the following.
# df = tcia.downloadSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_seedPoints")

#### Download 3D segmentations

In [ ]:
# filter dataframe to only include segmentations
segs = annotation_Metadata[annotation_Metadata['DICOM Type'] == 'SEG']
# display(segs)

# extract series UID column to list for downloading
series_data = segs["SeriesInstanceUID"].tolist()

# download a sample set of three scans 
# return metadata dataframe as dataframe
# save a CSV of the metadata 
df = tcia.downloadSampleSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_SEGs")

# If you'd rather download the full dataset, run the following.
# df = tcia.downloadSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_SEGs")

#### Download source images for seed points AND segmentations

In [ ]:
# filter dataframe to only include seg and seed point rows (remove "no findings")
ref_Series = annotation_Metadata[(annotation_Metadata['StructureSetLabel'] == 'Seed Points') |
                                 (annotation_Metadata['DICOM Type'] == 'SEG')]

# remove duplicate ReferencedSeriesUIDs
clean_refSeries = ref_Series.drop_duplicates(subset='ReferencedSeriesInstanceUID')
# display(clean_refSeries)

# extract series UID column to list for downloading
series_data = clean_refSeries["ReferencedSeriesInstanceUID"].tolist()

# download a sample set of three scans 
# return metadata dataframe as dataframe
# save a CSV of the metadata 
df = tcia.downloadSampleSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "seg_seed_source_images")

# If you'd rather download the full dataset, run the following.
# df = tcia.downloadSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_seg_seed_MRIs")

#### Download source images for segmentations (excludes MRIs that only had seed points without segmentations)

In [ ]:
# filter dataframe to only include seg
ref_Series = annotation_Metadata[annotation_Metadata['DICOM Type'] == 'SEG']
# display(ref_Series)

# remove duplicate ReferencedSeriesUIDs
clean_refSeries = ref_Series.drop_duplicates(subset='ReferencedSeriesInstanceUID')
# display(clean_refSeries)

# extract series UID column to list for downloading
series_data = clean_refSeries["ReferencedSeriesInstanceUID"].tolist()

# download a sample set of three scans 
# return metadata dataframe as dataframe
# save a CSV of the metadata 
df = tcia.downloadSampleSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_seg_MRIs")

# If you'd rather download the full dataset, run the following.
# df = tcia.downloadSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_seg_MRIs")

The following code will download the MRI scans for images with negative finding assessments.  These are cases where the authors of the dataset did not find anything that could be annotated.  Downloading these scans could be useful if you are training a tumor/metastases detection model.

In [ ]:
# filter dataframe to only include MRIs with "no findings"
ref_Series = annotation_Metadata[annotation_Metadata['StructureSetLabel'] == 'No Findings']

# remove duplicate ReferencedSeriesUIDs
clean_refSeries = ref_Series.drop_duplicates(subset='ReferencedSeriesInstanceUID')

# extract series UID column to list for downloading
series_data = clean_refSeries["ReferencedSeriesInstanceUID"].tolist()

# download a sample set of three scans
# return metadata dataframe as dataframe
# save a CSV of the metadata 
df = tcia.downloadSampleSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_noFinding_MRIs")

# If you'd rather download the full dataset, run the following.
# df = tcia.downloadSeries(series_data, api_url = "restricted", input_type = "list", csv_filename = "acns0332_noFinding_MRIs")

# Acknowledgements
[The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a service that de-identifies and hosts a large, publicly available, archive of medical images of cancer.  TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/), and is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/).

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/).  If you leverage TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). Upon receiving access, you must also abide by the terms of your NCTN/NCORP Data Archive’s Data Use Agreement (DUA). You are not allowed to redistribute the data or use it for other purposes. Attribution should include references to the following citations:

## Data Citations

1. Hwang, E. I., Kool, M., Burger, P. C., Capper, D., Chavez, L., Brabetz, S., Williams-Hughes, C., Billups, C., Heier, L., Jaju, A., Michalski, J., Li, Y., Leary, S., Zhou, T., von Deimling, A., Jones, D. T. W., Fouladi, M., Pollack, I. F., Gajjar, A., … Olson, J. M. (2021). Chemotherapy and Radiation Therapy in Treating Young Patients With Newly Diagnosed, Previously Untreated, High-Risk Medulloblastoma/PNET (ACNS0332) [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/TCIA.582B-XZ89
2. Rozenfeld, M., & Jordan, P. (2022). Annotations for Chemotherapy and Radiation Therapy in Treating Young Patients With Newly Diagnosed, Previously Untreated, High-Risk Medulloblastoma/PNET (ACNS0332-Tumor-Annotations) (Version 1) [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/D8A8-6252

## Publication Citation

Hwang, E. I., Kool, M., Burger, P. C., Capper, D., Chavez, L., Brabetz, S., Williams-Hughes, C., Billups, C., Heier, L., Jaju, A., Michalski, J., Li, Y., Leary, S., Zhou, T., von Deimling, A., Jones, D. T. W., Fouladi, M., Pollack, I. F., Gajjar, A., … Olson, J. M. (2018). Extensive Molecular and Clinical Heterogeneity in Patients With Histologically Diagnosed CNS-PNET Treated as a Single Entity: A Report From the Children’s Oncology Group Randomized ACNS0332 Trial. Journal of Clinical Oncology, 36(34), 3388–3395. https://doi.org/10.1200/jco.2017.76.4720. Epub ahead of print. PMID: 30332335.

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7